In [6]:
from src.plot_experiment2 import plot_report


fig = plot_report("../../reports/breast-cancer", "breast-cancer", store=False)
fig.show()

In [12]:
import os

import numpy as np
from src.nas.nas_params import NasParams


report_folder = "../../reports/vertebral"

population = NasParams.load_population(os.path.join(report_folder, "population.csv"))
if population is None:
    population = np.array([])

population

array([[6, 1, 5, 4, 0, 1, 3, 6, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [5, 1, 0, 0, 0, 7, 0, 2, 5, 0, 0, 0, 0, 0, 0, 0, 0],
       [6, 1, 2, 0, 0, 3, 3, 6, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [7, 0, 3, 4, 0, 0, 1, 7, 3, 0, 0, 0, 0, 0, 1, 0, 0]])

In [ ]:
from dataclasses import asdict

import pandas as pd

from src.nas.chromosome import ChromosomeConfig
from src.nas.mlp_chromosome import MLPChromosome
from src.nas.mlp_nas_problem import MlpNasProblem
from src.datasets.vertebral_dataset import VertebralDataset
from src.models.eval import NNArchitectureComplexityEvaluator


def find_acc(chromosome: np.ndarray, report_folder: str) -> float:
    models = os.listdir(os.path.join(report_folder, "models"))
    str_ch = "-".join([str(x) for x in chromosome])

    for model in models:
        if str_ch in model:
            return float(model.split("_")[0])

    raise ValueError(f"Didn't find a model for chromosome: {str_ch}")


def report_to_df(folder: str):
    population = NasParams.load_population(os.path.join(folder, "population.csv"))
    if population is None:
        raise ValueError("Population is empty or not found.")

    cfg = ChromosomeConfig(MLPChromosome)
    nas_params = NasParams(batch_size=32)
    nas_problem = MlpNasProblem(nas_params, VertebralDataset)

    data = []

    for raw_ch in population:
        ch = cfg.decode(raw_ch)
        acc = find_acc(raw_ch, folder)
        cost = NNArchitectureComplexityEvaluator(
            nas_problem.get_nn_params(ch)
        ).evaluate_complexity()

        data.append(
            {
                "acc": acc,
                "cost": cost,
                **asdict(ch),
            }
        )

    df = pd.DataFrame(data)
    return df


df = report_to_df(report_folder)
df

,acc,cost,in_bitwidth,hidden_layers,hidden_height1,hidden_bitwidth1,hidden_height2,hidden_bitwidth2,hidden_height3,hidden_bitwidth3,output_bitwidth,compression,activation,reste_o,parameters_quantization_mode,activation_binarization_mode,dropout,learning_rate,weight_decay
0,85.1613,216.0,7,1,24,5,6,2,12,7,1,NNParamsCompMode.BINARY,Activation.NONE,1.5,QMode.DET,QMode.DET,0.1,0.001,0.00001
1,75.8065,54.0,6,1,6,1,6,8,6,3,6,NNParamsCompMode.BINARY,Activation.NONE,1.5,QMode.DET,QMode.DET,0.0,0.001,0.00001
2,81.9355,72.0,7,1,8,1,6,4,12,7,1,NNParamsCompMode.BINARY,Activation.NONE,1.5,QMode.DET,QMode.DET,0.1,0.001,0.00001
3,61.6129,18.0,8,0,12,5,6,1,7,8,4,NNParamsCompMode.BINARY,Activation.NONE,1.5,QMode.DET,QMode.DET,0.1,0.001,0.00001


In [29]:
import plotly.express as px


def scatter_population(df: pd.DataFrame, *args, title="Title", **kwargs):
    fig = px.scatter(
        df,
        x="acc",
        y="cost",
        color="activation",
        symbol="compression",
        title=title,
        labels={
            "compression": "Compression",
            "activation": "Activation",
            "acc": "Accuracy (%)",
            "cost": "Model Complexity",
        },
        template="plotly_white",
        **kwargs,
    )
    fig.update_xaxes(autorange="reversed")
    fig.update_layout(margin=dict(autoexpand=True))

    return fig

scatter_population(df, title="Vertebral NAS Population")

In [38]:
report_name = "breast-cancer"
report_folder = "../../reports/" + report_name
df = report_to_df(report_folder)
fig = scatter_population(df, title=f"{report_name} NAS Population")
fig.show()